# Using Jupyter to write FastHTML

To write FastHTML applications in Jupyter notebooks requires a slightly different process than normal Python applications. 

:::{.callout-tip title='Download this notebook and try it yourself'}
The source code for this page is a [Jupyter notebook](https://github.com/AnswerDotAI/fasthtml/blob/main/nbs/tutorials/jupyter_and_fasthtml.ipynb). That makes it easy to directly experiment with it. However, as this is working code that means we have to comment out a few things in order for the documentation to build.
:::

## Getting started

The first step is to import necessary libraries. As using FastHTML inside a Jupyter notebook is a special case, it remains a special import. 

In [ ]:
from fasthtml.common import *
from fasthtml.jupyter import *

Let's use `jupy_app` to instantiate `app` and `rt` objects. `jupy_app` is a thin wrapper around the `fast_app` function.

In [ ]:
app, rt = jupy_app()

Define two view handlers to test the application.

In [ ]:
@rt
def index():
    return Titled('Hello, Jupyter',
        P(A('Click me to get a random number', hx_get=randomizer, hx_target='#result')),
        P(A('Reload this page', href=index)),   
        Div(id='result')                  
    )

In [ ]:
import random

@rt
def randomizer(): return Div(P(f'Random {random.randint(1,100)}'))

Set the `port` in a variable so it can be more easily shared between the `server` and `HTMX` symbols described below.

In [ ]:
port = 8000

Create a `server` object using `JupyUvi`, which also starts Uvicorn. The `server` runs in a separate thread from Jupyter, so it can use normal HTTP client functions in a notebook.

In [ ]:
# Run locally, uncomment this line to create and start the server
# server = JupyUvi(app, port=port)

The `HTMX` callable displays the server's HTMX application in an iframe which can be displayed by Jupyter notebook. Pass in the same `port` variable used in the `JupyUvi` callable above.

In [ ]:
# Run locally, uncomment this line to display the HTMX application in Jupyter
# HTMX(port=port)

## Refreshing handlers

Just like with normal web pages, if a handler is changed we need to refresh the rendered page to see the changes. In a web app `fast_app()` does this via a websocket, but inside Jupyter after running the cell on a handler we need to take an extra step of manually updating the rendered HTML. 

We can see this in action by following these steps:

1. Cange the value of 'Hello, Jupyter' in the `Titled()` element to `Hello, Jupyter + FastHTML`
2. click the 'Reload this page' link

You'll see the change you made appear inside the HTMX view. This happens because the line written to be `P(A('Reload this page', href=index)),` does just that - it reloads the page. 

:::{.callout-note title='When the HTMX() view must be refreshed'}
Sometimes we have to refresh the HTMX() page. This can happen when no means to refresh the page is provided. To perform this action run again the cell that contains `HTMX(port=port)`. 
:::

## Full screen view


You can view your app outside of Jupyter by going to `localhost:PORT`, where `PORT` is what you set the `port` variable to. This can be useful for debugging. Unless you change the `port` value in this tutorial, just click on this link: [localhost:8000/](http://localhost:8000/).

Understand that this is FastHTML running in Jupyter mode, so things will look different. For example, unlike `fast_app`, `jupy_app` doesn't include Picoweb CSS as a default.

## Graceful shutdowns

Use the `server.stop()` function displayed below. If you restart Jupyter without calling this line the thread may not be released and the `HTMX` callable above may throw errors. If that happens, a quick temporary fix is to change the `port` number above to something else.

Cleaner solutions to the dangling thread are to kill the dangling thread (dependant on each operating system) or restart the computer.

In [ ]:
# Run locally, uncomment this line to stop the server
# server.stop()